In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy
import skimage
# import custom functions
import sys
sys.path.append("../../../../")
import utils
import os
from tqdm import tqdm
import torch
from skimage.io import imread
from scipy.io import loadmat
from skimage.measure import regionprops
from skimage.transform import resize
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import re
import graph

code does not include cropping images

In [2]:
# load data
df = pd.read_csv('../../../../../data/chl/cHL/crop/OneDrive_1_2-27-2023/reg001_trim_annotations.csv',
                 index_col = 0)
df.head()

,X_cent,Y_cent,cellSize,cellType,BCL.2,CCR6,CD11b,CD11c,CD15,CD16,...,PD.L1,Podoplanin,T.bet,TCR.g.d,TCRb,Tim.3,VISA,Vimentin,a.SMA,b.Catenin
cellLabel,,,,,,,,,,,,,,,,,,,,,
1,75,2,42,B,0.792642,0.617038,0.000089,0.000040,0.000029,0.097688,...,0.091339,0.114237,0.241829,0.681727,0.159092,0.006533,0.000454,0.139826,0.0,0.057827
2,1767,2,63,DC,0.000000,0.127394,0.062173,0.660572,0.000000,0.000000,...,0.163511,0.000193,0.010475,0.231264,0.058234,0.111610,0.182849,0.899717,0.0,0.267198
3,2545,2,29,NK,0.060620,0.134818,0.298522,0.083348,0.000000,0.182152,...,0.170917,0.000193,0.033671,0.201099,0.000000,0.004900,0.454174,0.069180,0.0,0.104686
4,2899,3,64,Monocyte,0.000000,0.012896,0.287698,0.268121,0.000000,0.124361,...,0.052713,0.005982,0.000000,0.000000,0.052230,0.148087,0.228221,0.053541,1.0,0.283151
5,3657,3,61,CD4,0.201057,0.110590,0.000000,0.000000,0.000000,0.000000,...,0.026865,0.011771,0.124955,0.154981,0.597647,0.013284,0.000454,0.234038,0.0,0.000000


In [4]:
# save out
# feature edge
# feature label
# cell nbhd
# spatial edge

mapping = {'B':'B', 'DC':'DC', 'NK':'NK', 'Monocyte':'Monocyte', 
          'CD4':'CD4', 'Lymphatic':'Lymphatic', 'CD8':'CD8', 'M1':"Mph", 'Endothelial':'Endothelial',
          'Mast':'Mast', 'Tumor':'Tumor', 'Neutrophil':'Neutrophil', 'M2':"Mph", 'TReg':'Other', 
          'Cytotoxic CD8':'CD8', 'Epithelial':'Epithelial', 'Other':'Other', 'Seg Artifact':'Seg Artifact'}

df["cluster.term"] = df["cellType"].map(mapping)
df = df[df["cluster.term"] != 'Seg Artifact']
data_dir = '/mnt/cloud1/sheng-projects/st_projects/spatial_clust/spatial-clust-scripts/ipynb/Bokai_reorg/cHL_codex/data/'
df.to_csv(os.path.join(data_dir, 'features_and_metadata.csv'))


feature_names = [
        'CD2', 'CD56',
        'CD20', 'CD30', 'CD31', 'CD4', 
        'CD57',  'CD7', 'CD8',
        'Granzyme.B','FoxP3', 'CD11c', 'CD16', 'CD206', 'CD163', 'CD68', 'CD15', 'CD11b',
        'MCT', 'Cytokeratin', 
        'Podoplanin', 'a.SMA'
    ]

features = df[feature_names].to_numpy()
features = utils.center_scale(features)

res = 0.5
knn = 20

feature_edges = graph.get_feature_edges(
    arr=features, pca_components=20,
    n_neighbors=15, metric='correlation', verbose=False
)

feature_labels = graph.graph_clustering(
    df.shape[0], feature_edges, resolution=res, n_clusters=None, n_runs=1,
    resolution_tol=0.05, seed=None, verbose=False
)

np.save(os.path.join(data_dir, f'feature_labels_res{res}.npy'), feature_labels)
np.save(os.path.join(data_dir, f'feature_edges_res{res}.npy'), np.array(feature_edges[:2]).T)

print('Calculating cell neighborhood composition matrix...')
locations = df[['X_cent', 'Y_cent']].to_numpy()
spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=knn, method='kd_tree')
cell_nbhd = utils.get_neighborhood_composition(knn_indices=spatial_knn_indices, labels=feature_labels)
np.save(os.path.join(data_dir, f'cell_nbhd_res{res}_k{knn}.npy'), cell_nbhd)

## save out spatial edge, always 15 cells
locations = df[['X_cent', 'Y_cent']].to_numpy()
spatial_edges = graph.get_spatial_edges(
    arr=locations, n_neighbors=15, verbose=True
)
np.save(os.path.join(data_dir, f'spatial_edges_0325.npy'), np.array(spatial_edges[:2]).T)

/home/bokai/miniconda3/envs/cellsnap/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Calculating cell neighborhood composition matrix...


In [19]:
### added for concact

cell_nbhd = np.load('../data/cell_nbhd_res0.5_k20.npy')
feat = pd.read_csv('../data/features_and_metadata.csv')
features_names = [
        'CD2', 'CD56',
        'CD20', 'CD30', 'CD31', 'CD4', 
        'CD57',  'CD7', 'CD8',
        'Granzyme.B','FoxP3', 'CD11c', 'CD16', 'CD206', 'CD163', 'CD68', 'CD15', 'CD11b',
        'MCT', 'Cytokeratin', 
        'Podoplanin', 'a.SMA'
    ]
features = feat.loc[:,features_names].to_numpy()

feat_nbhd_concact = np.concatenate((cell_nbhd, features), axis = 1)#.shape

In [20]:
np.save('../data/nbhd_feat_concact.npy', feat_nbhd_concact)
print(feat_nbhd_concact.shape)

(143730, 39)


In [7]:
feat_nbhd_concact = np.concatenate((cell_nbhd, features), axis = 1)#.shape
np.save('../data/nbhd_feat_concact_human.npy', feat_nbhd_concact)
print(feat_nbhd_concact.shape)

(143730, 36)


Custom code to prepare the single cell cropped images:

In [ ]:
trun_qr = 0.9
pad = 1000
size = 80
n_channels = 2
load_path = '../../../data/chl/cHL/crop/OneDrive_1_2-27-2023/'
image_load_path = '../../../data/chl/cHL/'

save_folder = os.path.join(image_load_path, "processed_data", "single_cell_images", f"size{size}_channels{n_channels}_qr{trun_qr}_images")
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

n_cells = df.shape[0]

channel1 = imread(os.path.join(load_path, 'reg001_cropped/DAPI-01.tiff'))
channel2 = imread(os.path.join(load_path, 'reg001_cropped/CD30.tiff'))
channel3 = imread(os.path.join(load_path, 'reg001_cropped/CD20.tiff'))
channel4 = imread(os.path.join(load_path, 'reg001_cropped/Vimentin.tiff'))
pad_image = np.dstack([channel2, channel3, channel4, channel1])

print("plotting the nucl channel")
plt.imshow(pad_image[:, :, -1])
plt.show()


print("begin padding and truncating")
zero_pad_entire_view = np.zeros((pad_image.shape[0]+2*pad, pad_image.shape[1]+2*pad, pad_image.shape[2]))
zero_pad_entire_view[pad:pad_image.shape[0]+pad, pad:pad_image.shape[1]+pad, :] = pad_image
truncate = np.quantile(pad_image, q = trun_qr, axis = (0, 1))
#truncate[-1] = 4000
truncate = truncate[None, None, :]

zero_pad_entire_view[zero_pad_entire_view <= truncate] = 0
zero_pad_entire_view[zero_pad_entire_view > truncate] = 1

pad_images_sum = np.zeros((zero_pad_entire_view.shape[0], zero_pad_entire_view.shape[1], 2))
pad_images_sum[:, :, 1] = zero_pad_entire_view[:, :, -1]
pad_images_sum[:, :, 0] = (zero_pad_entire_view[:, :, 0] + zero_pad_entire_view[:, :, 1] + zero_pad_entire_view[:, :, 2])>0

print("Processing each cell...and saving!", flush = True)
for i in tqdm(range(n_cells)):
    # process each cell
    center_x = df.iloc[i]["Y_cent"] # not we switched xy here, this is just custom formatting difference for this specific dataset
    center_y = df.iloc[i]["X_cent"]
    cur_image = np.transpose(pad_images_sum[(int(center_x)-size//2+pad):(int(center_x)+size//2+pad), 
                                           (int(center_y)-size//2+pad):(int(center_y)+size//2+pad), 
                                           :], (2, 0, 1)).astype(np.int8)
    assert(cur_image.shape == (2, size, size))
    np.save(file = os.path.join(save_folder, f"img_{i:06d}"), arr = cur_image)
    if i % 100 == 99:
        plt.imshow(cur_image[0, :, :])
        plt.show()
        plt.imshow(cur_image[1, :, :])
        plt.show()
    